In [84]:
import json
import csv

In [11]:
# Converts Yao's CSV to a json
csv_path = 'data/input_yao.csv'
json_path = 'data/input_yao.json'

with open(csv_path, encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)

    out = []
    
    i = 0
    for row in csv_reader:
        if (i==0):
            print(row)
        i+=1
        
        out.append(row)
        
    with open(json_path, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(out, indent=4))

{'original': 'One side of the armed conflicts is composed mainly of the Sudanese military and the Janjaweed, a Sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan.', 'simplified': 'one side of the armed conflicts is composed of the sudanese military and the janjaweed , a sudanese militia group recruited from the afro-arab abbala tribes of the northern rizeigat in sudan .', 'system': 'editnts_w_split.txt'}


In [112]:
# Alignment is having trouble with long sentences
# Gets longest sentences
json_path = 'data/input_yao.json'

with open(json_path, encoding='utf-8') as f:
    in_data = json.load(f).copy()

for i in range(len(in_data)):
    in_data[i]['id'] = i

sorted_sents = sorted(in_data, key=lambda x: len(x['simplified']))

In [115]:
# Gets the list of sentences which haven't been aligned
in_no_exist = []
not_found = []
for i in range(len(sorted_sents)):
    sent = sorted_sents[i]['id']
    found = False
    for idx_pair in no_exist_idx:
        if sent >= idx_pair[0] and sent <= idx_pair[1]:
            in_no_exist.append(True)
            found = True
            not_found.append(sorted_sents[i])
            break
    if not found:
        in_no_exist.append(False)

In [94]:
# Checks to make sure alignments exist
import os
files = [x for x in os.listdir(os.getcwd() + "/data") if x.startswith("input_yao_aligned_")]
aligned_idxes = sorted([[int(x.replace(".json", "")) for x in f.split("_")[3:5]] for f in files])

curr_idx = 0
no_exist_idx = []
for i in range(len(aligned_idxes)-1):
    if (aligned_idxes[i][0] != curr_idx):
        no_exist_idx.append([curr_idx, aligned_idxes[i][0]])
        print(f"{curr_idx}-{aligned_idxes[i][0]}")
    curr_idx = aligned_idxes[i][1]

# Remember! There's no check to see if the same sentence has been processed twice, or if these overlap with each other, this won't work

90-100
150-200
275-300
325-350
400-425
650-675
700-750
775-800
950-975
1025-1050
1250-1275
1350-1375
1425-1525
1625-1700
1725-1800
2050-2100
2525-2550
3000-3025
3050-3075
3150-3175


In [ ]:
import json
aligned_json_path = 'data/input_yao_aligned.json'

In [73]:
# Combines alignements
combined = []
for path in files:
    with open("data/" + path, encoding='utf-8') as f:
        combined.extend(json.load(f).copy())

combined = sorted(combined, key=lambda x: x['id'])

with open(aligned_json_path, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(combined))

In [80]:
import json
aligned_formatted_json_path = 'data/input_yao_formatted.json'

In [81]:
# Convert's output json to Yao's requested format
with open(aligned_json_path, encoding='utf-8') as f:
    aligned = json.load(f)

In [82]:
# format: [edit_category, start_index, end_index, edit_id_in_that_category]
# 'deletion': 0, 'paraphrase': 1, 'split': 2, 'insertion': 3

newly_aligned = []

for sent_idx in range(len(aligned)):
    newly_aligned_entries = []
    
    insertion_count = 0
    deletion_count = 0
    paraphrase_count = 0
    
    original_spans = []
    simplified_spans = []
    
    for entry in aligned[sent_idx]['spans']:
        if (entry[0] == None):
            # insertion
            simplified_spans.append([3, entry[1][0], entry[1][1], insertion_count])
            insertion_count += 1
        elif (entry[1] == None):
            # deletion
            original_spans.append([0, entry[0][0], entry[0][1], deletion_count])
            deletion_count += 1
        else:
            # paraphrase
            original_spans.append([1, entry[0][0], entry[0][1], paraphrase_count])
            simplified_spans.append([1, entry[1][0], entry[1][1], paraphrase_count])
            paraphrase_count += 1
            
    newly_aligned_entries = aligned[sent_idx].copy()
    newly_aligned_entries['original_spans'] = original_spans
    newly_aligned_entries['simplified_spans'] = simplified_spans
    del newly_aligned_entries['spans']
    newly_aligned.append(newly_aligned_entries)

newly_aligned = sorted(newly_aligned, key=lambda x: x['id'])

In [83]:
with open(aligned_formatted_json_path, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(newly_aligned))

In [72]:
# Sanity check: return missing IDs
ids = [x['id'] for x in newly_aligned]
noalignment = []
for i in range(newly_aligned[-1]['id']):
    if i not in ids:
        noalignment.append(i)

print(f"Don't have alignments for: {noalignment}")

Don't have alignments for: [90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 73